In [19]:
!wget -q https://raw.githubusercontent.com/fuzhenxin/textstyletransferdata/master/sentiment/neg.txt
!wget -q https://raw.githubusercontent.com/fuzhenxin/textstyletransferdata/master/sentiment/pos.txt
!wget -q "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"

In [20]:
import sys
import numpy as np
import pandas as pd
import string
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
pos_text = pd.read_csv("pos.txt", delimiter='\t')
neg_text = pd.read_csv("neg.txt", delimiter='\t')

In [22]:
pos_text.columns = ['text']
pos_text['label'] = 1
neg_text.columns = ['text']
neg_text['label'] = 0

In [23]:
corpus = pd.concat([pos_text, neg_text])
corpus.reset_index(inplace=True)
del corpus['index']

In [24]:
corpus['text'] = corpus['text'].str.replace('[^\w\s]', '')
corpus['text'] = corpus['text'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [25]:
corpus['text'] = corpus['text'].apply(word_tokenize)

In [32]:
corpus_nostopwords = corpus.copy()

In [33]:
stop_words = []
with open("NLTK's list of english stopwords") as f:
  for line in f:
    stop_words.append(line.split('\n')[0])

In [34]:
corpus_nostopwords['text'] = corpus_nostopwords['text'].apply(lambda x: [item for item in x if item not in stop_words])
corpus_nostopwords['text'] = corpus_nostopwords['text'].apply(lambda x: " ".join(x))
corpus['text'] = corpus['text'].apply(lambda x: " ".join(x))
corpus_nostopwords['text']

0               already tried one recipe day receiving book
1          bought zoku quick pop daughterr zoku quick maker
2                                  loves fun make ice cream
3                                               hoping came
4                book emphasizes sweet dessert pops however
                                ...                        
799108           said whoa im gon na like gameunfortunately
799109    towards end first third game became insanely d...
799110    cases would finally find hiding point behind r...
799111                              die die die die die die
799112    well get pictureif dont like game usually put ...
Name: text, Length: 799113, dtype: object

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(corpus_nostopwords['text'], corpus_nostopwords['label'], test_size=0.2, random_state=42)

Unigrams no stop words

In [46]:
x_test = x_test[:int(len(x_test) * 0.5)]
x_val = x_test[int(len(x_test) * 0.5):]
y_test = y_test[:int(len(y_test) * 0.5)]
y_val = y_test[int(len(y_test) * 0.5):]

count_vect = CountVectorizer()
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train_t, y_train)

file_1 = 'mnb_uni_ns'

with open(file_1, 'wb') as f:
  pickle.dump((model, count_vect), f)

print(model.score(x_test_t, y_test))

0.8078087848829933


Unigrams no stop words

In [47]:
count_vect = CountVectorizer(ngram_range=(2, 2))
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

model = MultinomialNB()
model.fit(x_train_t, y_train)

file_2 = 'mnb_bi_ns'
with open(file_2, 'wb') as f:
  pickle.dump((model, count_vect), f)

print(model.score(x_test_t, y_test))

0.8242022275059442


Unigram+Bigram no stop words


In [48]:
count_vect = CountVectorizer(ngram_range=(1, 2))
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

model = MultinomialNB()
model.fit(x_train_t, y_train)

file_3 = 'mnb_uni_bi_ns'
with open(file_3, 'wb') as f:
  pickle.dump((model, count_vect), f)

print(model.score(x_test_t, y_test))

0.8320360405456139


Unigram with stop words



In [49]:
x_train, x_test, y_train, y_test = train_test_split(corpus['text'], corpus['label'], test_size=0.2, random_state=42)

x_test = x_test[:int(len(x_test) * 0.5)]
x_val = x_test[int(len(x_test) * 0.5):]
y_test = y_test[:int(len(y_test) * 0.5)]
y_val = y_test[int(len(y_test) * 0.5):]

count_vect = CountVectorizer()
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

model = MultinomialNB()
model.fit(x_train_t, y_train)
print(model.score(x_test_t, y_test))

file_4 = 'mnb_uni'
with open(file_4, 'wb') as f:
  pickle.dump((model, count_vect), f)

0.8078737595575077


Bigram with stop words

In [50]:
count_vect = CountVectorizer(ngram_range=(2, 2))
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

model = MultinomialNB()
model.fit(x_train_t, y_train)

file_5 = 'mnb_bi'
with open(file_5, 'wb') as f:
  pickle.dump((model, count_vect), f)

print(model.score(x_test_t, y_test))

0.8248176095906696


Unigram+Bigram with stop words


In [51]:
count_vect = CountVectorizer(ngram_range=(1, 2))
count_vect.fit(x_train)
x_train_t = count_vect.transform(x_train)
x_test_t = count_vect.transform(x_test)
x_tval_t = count_vect.transform(x_val)

model = MultinomialNB()
model.fit(x_train_t, y_train)

file_6 = 'mnb_uni_bi'
with open(file_6, 'wb') as f:
  pickle.dump((model, count_vect), f)

print(model.score(x_test_t, y_test))

0.8319004892943399


Tuning the model

In [52]:
from sklearn.model_selection import GridSearchCV
parameters = {'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}
grid_search = GridSearchCV(model, parameters)
grid_search.fit(x_tval_t, y_val)

grid_search.score(x_test_t, y_test)

0.8644867414999187